In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# === CONFIG ===
look_back = 24
data_dir = r"D:\8th SEMESTER\ML LAB\ML Lab Spring 25\ML Open ended lab"
target_col = 0  # Index of the target column

# === Load Data ===
train = pd.read_csv(os.path.join(data_dir, "train_data.csv"))
val = pd.read_csv(os.path.join(data_dir, "val_data.csv"))
test = pd.read_csv(os.path.join(data_dir, "test_data.csv"))

# === Prepare Sequences ===
def create_sequences(data, look_back, target_col):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data.iloc[i:i+look_back].values)
        y.append(data.iloc[i+look_back, target_col])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train, look_back, target_col)
X_val, y_val = create_sequences(val, look_back, target_col)
X_test, y_test = create_sequences(test, look_back, target_col)

# === Build LSTM Model ===
model = Sequential([
    LSTM(64, activation='tanh', input_shape=(look_back, X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# === Train ===
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# === Evaluate ===
val_loss = model.evaluate(X_val, y_val)
test_loss = model.evaluate(X_test, y_test)
print(f"Validation RMSE: {np.sqrt(val_loss):.4f}")
print(f"Test RMSE: {np.sqrt(test_loss):.4f}")

# === Save Model ===
model.save(os.path.join(data_dir, "lstm_model.h5"))
print("✅ LSTM model saved.")


c:\Users\Hassan Laptop Point\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - loss: 0.0101 - val_loss: 0.0041
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 4.9560e-04 - val_loss: 0.0028
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 2.5132e-04 - val_loss: 0.0023
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 1.4882e-04 - val_loss: 0.0020
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.1164e-04 - val_loss: 0.0019
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 8.1107e-05 - val_loss: 0.0016
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 5.7975e-05 - val_loss: 0.0016
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 4.9554e-05 - val_loss: 0.0015
Epoch 9/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 4.1882e-05 - val_loss: 0.0012
Epoch 10/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 3.4529e-05 - val_loss: 0.0012
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 6.5383e-04
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms

Validation RMSE: 0.0340
Test RMSE: 0.0251
✅ LSTM model saved.
